In [1]:
import sys
import polars as pl
import plotly.express as px
sys.path.insert(0, '..')
from fs_thesis import sql, show

# Create Overview for Tables Patients and Admissions

# Patient Demographics & Time-to-Event Extraction

Extracts patient demographic data and calculates time-to-event metrics for survival analysis.

## Data Structure
- **Unique Identifier**: `subject_id`
- **Demographics**: gender, age, insurance, language, marital status, race
- **Baseline**: First admission timestamp (`t0_time`)
- **Event**: ICD-9/10 diagnosis code occurrence
- **Censoring**: Date of death (`dod`) or end of follow-up

## Output
Generates a dataset with:
- Demographic features
- Time to event (days from baseline)
- Event indicator (occurred/censored)
- Target classes: Early event (<1yr), late event (>1yr), or censored

In [2]:
from fs_thesis.data_loader import load_final_data
df = load_final_data()
df.head().sort("subject_id")


subject_id,gender,anchor_age,dod,t0_time,insurance,language,marital_status,race,admission_type,event_time,t_event,t_death,event_occurred,duration,target
i64,str,i64,date,datetime[μs],str,str,str,str,str,datetime[μs],i64,i64,i32,i64,i32
10000947,"""M""",60,2121-08-23,2121-05-09 10:02:00,"""Private""","""English""","""MARRIED""","""BLACK/AFRICAN AMERICAN""","""EW EMER.""",null,null,105,0,105,2
10002167,"""F""",34,null,2163-09-04 08:00:00,"""Private""","""English""","""SINGLE""","""WHITE""","""ELECTIVE""",null,null,null,0,2000,2
10002266,"""F""",31,null,2131-02-28 14:57:00,"""Private""","""English""","""MARRIED""","""WHITE""","""URGENT""",null,null,null,0,2000,2
10004457,"""M""",65,null,2140-09-17 13:41:00,"""Medicare""","""English""","""SINGLE""","""WHITE""","""AMBULATORY OBSERVATION""",2148-09-14 14:19:00,2919,null,1,2919,1
10004733,"""M""",51,null,2174-12-04 11:28:00,"""Medicaid""","""English""","""SINGLE""","""UNKNOWN""","""URGENT""",null,null,null,0,2000,2


In [ ]:
# Prüfen, ob alle subject_ids einzigartig sind

print(is_unique)  # True oder False

True


# Event Time

Extracts the first occurrence of the target diagnosis (ICD-50 code, Beispiel: Herzinsuffizienz) for each patient.

Returns:
- `subject_id`: Patient identifier
- `event_time`: Admission timestamp of first diagnosis occurrence

# Time-to-Event Calculation & Target Definition

Calculates time intervals and creates target classes based on event occurrence and timing.

## Steps:
1. **Join datasets**: Merges baseline demographics with event times (left join)
2. **Calculate time differences**: 
   - `t_event`: Days from baseline to diagnosis
   - `t_death`: Days from baseline to death
3. **Determine duration**: Uses event time if available, otherwise death time, fallback to 2000 days
4. **Handle errors**: Clips negative durations to 0
5. **Create target classes**:
   - Class 0: Early event (<365 days)
   - Class 1: Late event (>365 days)
   - Class 2: Censored (no event)

# Feature & Target Extraction

Prepares data for machine learning by selecting demographic features and target variable.

## Output:
- **X**: Feature matrix with demographics (pandas DataFrame)
  - gender, anchor_age, insurance, language, marital_status, race, admission_type
- **y**: Target vector with class labels (numpy array)
  - 0: Early event (<365 days)
  - 1: Late event (>365 days)
  - 2: Censored (no event)

In [3]:
# Feature Engineering: BMI
# Strategie: Wir nutzen den vorgefertigten BMI Wert, da er eine höhere Abdeckung hat als (Weight & Height) kombiniert.
# Aggregation: Wir nehmen den Median pro Patient, um Ausreißer zu minimieren.

df_bmi = sql("""
    SELECT 
        subject_id,
        MEDIAN(CAST(result_value AS FLOAT)) as bmi
    FROM hosp.omr
    WHERE result_name = 'BMI (kg/m2)'
    -- Einfache Bereinigung: Nur plausible Werte (z.B. 10 bis 100)
    AND TRY_CAST(result_value AS FLOAT) BETWEEN 10 AND 100
    GROUP BY subject_id
""")

# Check distribution
show(df_bmi, limit=True)

,subject_id,bmi
0,10772290,34.950001
1,10772409,63.000000
2,10780398,37.049999
3,10781168,27.150000
4,10789772,29.299999
...,...,...
153536,19479574,39.450001
153537,19479949,27.700001
153538,19480145,30.299999
153539,19480286,33.500000


In [4]:
# Join to clean dataset
# Wir machen einen Left Join, da nicht alle Patienten einen BMI haben (Missing Values werden später von TabPFN handled!)
df_final = df_final.join(
    df_bmi, 
    on="subject_id", 
    how="left"
)

# Prüfen wie viele Missings wir haben
print(f"Missing BMI: {df_final['bmi'].null_count()} of {len(df_final)}")
show(df_final.select(['subject_id', 'anchor_age', 'bmi', 'target']), limit=True)

NameError: name 'df_final' is not defined

In [ ]:
import polars as pl
from sklearn.model_selection import train_test_split

# 1. Konvertierung für Scikit-Learn (Split-Logik)
df_pd = df_final.to_pandas()

# Erster Split: Trenne das finale Test-Set ab (20% Hold-out)
df_train_val_raw, df_test = train_test_split(
    df_pd, 
    test_size=0.2, 
    stratify=df_pd["target"], 
    random_state=42
)

# Zweiter Split: Trenne den Rest in Training und Validierung (z.B. 80/20 vom Rest)
df_train_raw, df_val = train_test_split(
    df_train_val_raw, 
    test_size=0.2, 
    stratify=df_train_val_raw["target"], 
    random_state=42
)

# --- BACK TO POLARS: Balancing nur für das TRAINING ---
# Wir nehmen 3.000 Samples pro Klasse, um innerhalb des TabPFN-Limits zu bleiben
df_train_pl = pl.from_pandas(df_train_raw)
n_samples = 3000

df_balanced_train = pl.concat([
    df_train_pl.filter(pl.col("target") == 0).sample(n=min(n_samples, df_train_pl.filter(pl.col("target")==0).height), seed=42),
    df_train_pl.filter(pl.col("target") == 1).sample(n=min(n_samples, df_train_pl.filter(pl.col("target")==1).height), seed=42),
    df_train_pl.filter(pl.col("target") == 2).sample(n=n_samples, seed=42)
]).sample(fraction=1.0, shuffle=True, seed=42)

# --- FINALE FEATURES & TARGETS ---
feature_cols = ["gender", "anchor_age", "insurance", "language", "marital_status", "race", "admission_type", "bmi"]

# Training (Balanced)
X_train = df_balanced_train.select(feature_cols).to_pandas()
y_train = df_balanced_train.select("target").to_series().to_numpy()

# Validation (Realistisch, nicht balanciert)
X_val = df_val[feature_cols]
y_val = df_val["target"].values

# Test (Realistisch, nicht balanciert - Hold-out)
X_test = df_test[feature_cols]
y_test = df_test["target"].values

print(f"Train (balanced): {len(y_train)} | Val (real): {len(y_val)} | Test (real): {len(y_test)}")

KeyboardInterrupt: 

> ℹ️ **INFO**  
> Why Train set is smaller than val and test. The Train set is balanced and gives a total of 9k for the TabPFN Algo. The Test and Val set is larger because it's a % of the total dataset (200k)

# Tab PFN
- training

In [ ]:
# 1. Training-Set massiv verkleinern (für Speed & Datenschutz-Konformität)
# Wir nehmen nur 333 Samples pro Klasse = ca. 1000 Total
n_emergency = 333

df_emergency_train = pl.concat([
    df_train_pl.filter(pl.col("target") == 0).sample(n=min(n_emergency, df_train_pl.filter(pl.col("target")==0).height)),
    df_train_pl.filter(pl.col("target") == 1).sample(n=min(n_emergency, df_train_pl.filter(pl.col("target")==1).height)),
    df_train_pl.filter(pl.col("target") == 2).sample(n=n_emergency)
]).sample(fraction=1.0, shuffle=True)

X_train_fast = df_emergency_train.select(feature_cols).to_pandas()
y_train_fast = df_emergency_train.select("target").to_series().to_numpy()

# 2. Classifier neu initialisieren (CPU ist bei 1000 Samples oft stabiler als ein buggy MPS-Backend)
from tabpfn import TabPFNClassifier
classifier = TabPFNClassifier(device='cpu') # Zurück auf CPU, da N jetzt klein ist

# 3. Fit (Sekundensache)
classifier.fit(X_train_fast, y_train_fast)

# 4. Vorhersage für 500 Test-Patienten (ohne Batches, einfach direkt)
X_test_fast = X_val.iloc[:500]
y_test_fast = y_val[:500]

print("Starte Express-Vorhersage...")
y_val_pred_fast = classifier.predict(X_test_fast)
print("Fertig!")

/Users/andrey/Repositories/fs-thesis/.venv/lib/python3.13/site-packages/tabpfn/validation.py:56: UserWarning: Running on CPU with more than 200 samples may be slow.
Consider using a GPU or the tabpfn-client API: https://github.com/PriorLabs/tabpfn-client
  _validate_num_samples_for_cpu(


Starte Express-Vorhersage...
Fertig!


# Evaluation der TabPFN Analyse

In [ ]:
import plotly.figure_factory as ff
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd

# 1. Classification Report ausgeben (Text-Form für die harten Fakten)
print("EXPRESS-CHECK REPORT (N_train=1000):")
report_dict = classification_report(y_test_fast, y_val_pred_fast, 
                                   target_names=['Früh (<1J)', 'Spät (1-3J)', 'Gesund'],
                                   output_dict=True,
                                   zero_division=0)
print(classification_report(y_test_fast, y_val_pred_fast, 
                           target_names=['Früh (<1J)', 'Spät (1-3J)', 'Gesund']))

import plotly.figure_factory as ff
import numpy as np

# 1. Normalisierung der Confusion Matrix (Zeilenweise)
# Wie viel % der tatsächlichen Klasse wurden wie vorhergesagt?
cm = confusion_matrix(y_test_fast, y_val_pred_fast)
cm_perc = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

labels = ['Früh (<1J)', 'Spät (1-3J)', 'Gesund']

# 2. Text für die Boxen (Absolute Zahl + Prozent)
annot_text = [
    [f"<b>{val}</b><br>({perc:.1%})" for val, perc in zip(row_val, row_perc)]
    for row_val, row_perc in zip(cm, cm_perc)
]

# 3. Interaktive Heatmap
fig = ff.create_annotated_heatmap(
    cm_perc, 
    x=labels, 
    y=labels, 
    annotation_text=annot_text, 
    colorscale='Reds' # 'Reds' hebt die Treffer besser hervor
)

fig.update_layout(
    title='Zusammenhang: Soziale Faktoren & Wiederaufnahme (Normalisiert)',
    xaxis_title="Vorhersage des Modells",
    yaxis_title="Tatsächlicher Verlauf (MIMIC-Daten)",
    template="plotly_white",
    height=600
)

fig.show()

EXPRESS-CHECK REPORT (N_train=1000):
              precision    recall  f1-score   support

  Früh (<1J)       0.10      0.52      0.17        21
 Spät (1-3J)       0.08      0.87      0.15        15
      Gesund       0.98      0.50      0.66       464

    accuracy                           0.51       500
   macro avg       0.39      0.63      0.33       500
weighted avg       0.92      0.51      0.63       500



In [ ]:
import plotly.graph_objects as go

# Daten aus deiner Confusion Matrix extrahieren
# Reihenfolge: [Früh, Spät, Gesund]
cm = confusion_matrix(y_test_fast, y_val_pred_fast)

# Labels für die Knoten
label_list = [
    "Tatsächlich: Früh", "Tatsächlich: Spät", "Tatsächlich: Gesund", # Quellen (Links)
    "Vorhergesagt: Früh", "Vorhergesagt: Spät", "Vorhergesagt: Gesund" # Ziele (Rechts)
]

# Definition der Flüsse (Sankey-Struktur)
source = [0, 0, 0, 1, 1, 1, 2, 2, 2] # Index der Quellen
target = [3, 4, 5, 3, 4, 5, 3, 4, 5] # Index der Ziele
value = cm.flatten() # Die Zahlen aus deiner Matrix

# Farben definieren (Grün für korrekt, Rot für Fehler)
color_link = [
    'rgba(31, 119, 180, 0.4)', 'rgba(31, 119, 180, 0.2)', 'rgba(31, 119, 180, 0.1)', # Von Früh
    'rgba(255, 127, 14, 0.2)', 'rgba(255, 127, 14, 0.4)', 'rgba(255, 127, 14, 0.1)', # Von Spät
    'rgba(44, 160, 44, 0.1)', 'rgba(44, 160, 44, 0.1)', 'rgba(44, 160, 44, 0.4)'    # Von Gesund
]

fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15, thickness = 20, line = dict(color = "black", width = 0.5),
      label = label_list, color = "blue"
    ),
    link = dict(
      source = source, target = target, value = value, color = color_link
  ))])

fig.update_layout(title_text="Patienten-Fluss: Realität vs. KI-Vorhersage", font_size=12)
fig.show()

In [ ]:
from sklearn.inspection import permutation_importance
import pandas as pd
import plotly.express as px

print("Berechne Feature Importance (kann ca. 1-2 Min. dauern oder 5)...")

# Wir nutzen unser Express-Testset für die Berechnung
result = permutation_importance(
    classifier, X_test_fast, y_test_fast, n_repeats=10, random_state=42, n_jobs=1
)

# Ergebnisse in ein DataFrame gießen
importance_df = pd.DataFrame({
    'Feature': feature_cols,
    'Importance': result.importances_mean,
    'Std_Dev': result.importances_std
}).sort_values(by='Importance', ascending=True) # Aufsteigend für horizontalen Plot

# Plotly Bar Chart
fig = px.bar(
    importance_df, 
    x='Importance', 
    y='Feature', 
    orientation='h',
    title='Welche sozialen Faktoren beeinflussen die Wiederaufnahme?',
    labels={'Importance': 'Wichtigkeit (Mean Decrease Accuracy)'}, # Wissenschaftlicher Label
    error_x='Std_Dev', # Zeigt die Variabilität der Wichtigkeit
    template="plotly_white",
    color='Importance',
    color_continuous_scale='Reds'
)

# NEU: Prozent-Formatierung für bessere Lesbarkeit
fig.update_layout(height=500, xaxis_tickformat='.1%')
fig.show()

Berechne Feature Importance (kann ca. 1-2 Min. dauern oder 5)...


In [ ]:
# Check: Zusammenhang zwischen Alter und Versicherung (Medicare)
# Wir schauen, wie hoch das Durchschnittsalter pro Versicherungsgruppe ist.
import plotly.express as px

df_check = X_test_fast.copy() # Kopie für Analyse
if hasattr(df_check, "to_pandas"):
    df_check = df_check.to_pandas()

# Boxplot zeigt klar: Medicare-Patienten sind fast alle 65+
fig = px.box(df_check, x="insurance", y="anchor_age", 
             title="Warum Alter unwichtig wirkt: Der 'Medicare'-Effekt",
             points="all", 
             color="insurance")
fig.show()

# Detaillierte Risiko-Analyse

Hier untersuchen wir, welche spezifischen Gruppen (Alter, BMI, Geschlecht) das höchste Risiko für einen **frühen Krankheitsausbruch (<1 Jahr)** tragen.

Wir nutzen die vom Modell vorhergesagte Wahrscheinlichkeit (`predict_proba`) für Klasse 0 (Early Event) als **Risk Score**.

In [ ]:
df_analyze = X_test_fast.copy()
if hasattr(df_analyze, "to_pandas"):
    df_analyze = df_analyze.to_pandas()

print(f"Analyse basierend auf {len(df_analyze)} Patienten aus dem Test-Set.")

Analyse basierend auf 500 Patienten aus dem Test-Set.


In [ ]:
import pandas as pd
import plotly.express as px

# 1. Daten vorbereiten (Scores berechnen)
y_proba_fast = classifier.predict_proba(X_test_fast)
# Wahrscheinlichkeit für Klasse 0 (Early Event)
risk_score = y_proba_fast[:, 0]

df_analyze['risk_score'] = risk_score
df_analyze['true_label'] = y_test_fast


In [ ]:
# ---------------------------------------------------------
# 2. BMI Analyse (Vereinfacht: Balkendiagramm)
# ---------------------------------------------------------
# Wir schauen uns nur den DURCHSCHNITT an, das ist einfacher zu lesen als Boxplots.
bins = [0, 18.5, 25, 30, 100]
labels = ['Untergewicht (<18.5)', 'Normal (18.5-25)', 'Übergewicht (25-30)', 'Adipositas (>30)']
df_analyze['bmi_group'] = pd.cut(df_analyze['bmi'], bins=bins, labels=labels)

df_bmi_mean = df_analyze.groupby('bmi_group', observed=True)['risk_score'].mean().reset_index()

fig1 = px.bar(
    df_bmi_mean, 
    x='bmi_group', 
    y='risk_score',
    text_auto='.1%', # Zeigt %-Wert direkt auf dem Balken
    title='Durchschnittliches Risiko nach BMI-Gruppe',
    labels={'risk_score': 'Wahrscheinlichkeit (Früher Ausbruch)', 'bmi_group': 'BMI Gruppe'},
    color='risk_score',
    color_continuous_scale='Reds',
    template="plotly_white"
)
fig1.update_layout(yaxis_tickformat='.0%') # Y-Achse als Prozent formatieren
fig1.show()

In [ ]:
# ---------------------------------------------------------
# 3. Alters Analyse (Vereinfacht)
# ---------------------------------------------------------
# Bins erweitert, um alle Altersgruppen zu erfassen (0-10, 10-20, ..., 90+)
df_analyze['age_group'] = pd.cut(
    df_analyze['anchor_age'], 
    bins=[0, 20, 30, 40, 50, 60, 70, 80, 90, 120],  # 0-20, 20-30, ..., 90-120
    labels=['<20', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80-89', '90+']
)
df_analyze['age_group'] = df_analyze['age_group'].astype(str)

df_age_mean = df_analyze.groupby('age_group')['risk_score'].mean().reset_index()

fig2 = px.bar(
    df_age_mean, 
    x='age_group', 
    y='risk_score',
    text_auto='.1%',
    title='Risiko-Entwicklung über das Alter',
    labels={'risk_score': 'Wahrscheinlichkeit (Früher Ausbruch)', 'age_group': 'Altersgruppe'},
    color='risk_score',
    color_continuous_scale='Reds',
    template="plotly_white"
)
fig2.update_layout(yaxis_tickformat='.0%')
fig2.show()

In [ ]:
# ---------------------------------------------------------
# 4. Sozio-Ökonomisch (Vereinfacht: Balken nebeneinander)
# ---------------------------------------------------------
# Statt komplizierter Violin-Plots nutzen wir gruppierte Balken.
df_ins_mean = df_analyze.groupby(['insurance', 'gender'])['risk_score'].mean().reset_index()

fig3 = px.bar(
    df_ins_mean, 
    x='insurance', 
    y='risk_score', 
    color='gender', 
    barmode='group', # Männlich/Weiblich nebeneinander
    text_auto='.1%',
    title='Vergleich: Wer hat das höchste Risiko? (Versicherung & Geschlecht)',
    labels={'risk_score': 'Wahrscheinlichkeit (Ø)', 'insurance': 'Versicherung'},
    template="plotly_white"
)
fig3.update_layout(yaxis_tickformat='.0%')
fig3.show()

# Robustness Experiment (Stabilitätstest)

Um zu beweisen, dass die Ergebnisse kein Zufall sind, führen wir das Training **30 Mal** mit unterschiedlichen Seeds durch.

**Ablauf:**
1. Ziehe in jedem Durchlauf ein **neues, balanciertes Training-Set** (Sampling Variation).
2. Trainiere ein frisches TabPFN Modell.
3. Evaluiere auf einem **fixen Validierungs-Set**.
4. Speichere Metriken (F1-Score, Accuracy) in einer CSV und visualisiere die Varianz.

Dies simuliert, wie robust das Modell gegenüber Veränderungen in den Trainingsdaten ist.

In [ ]:
import os
import time
import datetime
import pandas as pd
import numpy as np
import polars as pl
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.inspection import permutation_importance
from tabpfn import TabPFNClassifier

# 1. Konfiguration
IS_TEST_RUN = True  # KIPPSCHALTER
TIMESTAMP = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

if IS_TEST_RUN:
    N_LOOPS = 1
    BASE_DIR = "../../reports/robustness_experiment_test"
    print(f"⚠️ TEST-LAUF: Starte nur {N_LOOPS} Runde.")
else:
    N_LOOPS = 30
    BASE_DIR = "../../reports/robustness_experiment"
    print(f"🚀 PRODUKTIV-LAUF: Starte {N_LOOPS} Runden.")

RESULTS_DIR = os.path.join(BASE_DIR, TIMESTAMP)
os.makedirs(RESULTS_DIR, exist_ok=True)

X_val_robust = X_val.iloc[:1000].copy()
y_val_robust = y_val[:1000].copy()
results_list = []

print(f"🚀 Starte Experiment. Ergebnisse landen in: {RESULTS_DIR}")

# 2. Experiment Loop
for i in range(N_LOOPS):
    start_time = time.time()
    
    # Ordner für diesen Run
    run_dir = os.path.join(RESULTS_DIR, f"run_{i}")
    os.makedirs(run_dir, exist_ok=True)
    
    # A. Resampling
    current_seed = 42 + i
    n_per_class = 333
    
    df_iter_train = pl.concat([
        df_train_pl.filter(pl.col("target") == 0).sample(n=min(n_per_class, df_train_pl.filter(pl.col("target")==0).height), seed=current_seed),
        df_train_pl.filter(pl.col("target") == 1).sample(n=min(n_per_class, df_train_pl.filter(pl.col("target")==1).height), seed=current_seed),
        df_train_pl.filter(pl.col("target") == 2).sample(n=n_per_class, seed=current_seed)
    ]).sample(fraction=1.0, shuffle=True, seed=current_seed)
    
    X_train_iter = df_iter_train.select(feature_cols).to_pandas()
    y_train_iter = df_iter_train.select("target").to_series().to_numpy()
    
    # B. Training
    clf = TabPFNClassifier(device='cpu') 
    clf.fit(X_train_iter, y_train_iter)
    
    # C. Prediction
    y_pred = clf.predict(X_val_robust)
    y_proba = clf.predict_proba(X_val_robust)
    
    # Metriken
    f1_macro = f1_score(y_val_robust, y_pred, average='macro')
    acc = accuracy_score(y_val_robust, y_pred)
    prec_macro = precision_score(y_val_robust, y_pred, average='macro', zero_division=0)
    rec_macro = recall_score(y_val_robust, y_pred, average='macro', zero_division=0)
    f1_per_class = f1_score(y_val_robust, y_pred, average=None)

    # Feature Importance
    perm_result = permutation_importance(clf, X_val_robust, y_val_robust, n_repeats=2, random_state=42, n_jobs=1)
    
    duration = time.time() - start_time

    # --- RISK DIRECTION ANALYSE ---
    # Wir speichern durchschnittliche Risikoscores pro Gruppe in die CSV
    df_risk_check = X_val_robust.copy()
    df_risk_check['risk_score'] = y_proba[:, 0] # Klasse 0 = Early Event

    risk_stats = {}
    
    # 1. Kategorische Features (Gender, Insurance, etc.)
    cat_feats = [c for c in feature_cols if c not in ['bmi', 'anchor_age']]
    for cf in cat_feats:
        means = df_risk_check.groupby(cf, observed=True)['risk_score'].mean()
        for cat_val, val in means.items():
            # CSV-freundliche Spaltennamen
            safe_name = str(cat_val).replace(' ', '_').replace('/', '_').replace('-', '_').lower()[:20]
            risk_stats[f'risk_{cf}_{safe_name}'] = val

    # 2. Numerische Features (Bins)
    # BMI
    df_risk_check['bmi_grp'] = pd.cut(df_risk_check['bmi'], bins=[0, 18.5, 25, 30, 100], labels=['under', 'norm', 'over', 'obese'])
    for cat_val, val in df_risk_check.groupby('bmi_grp', observed=True)['risk_score'].mean().items():
        risk_stats[f'risk_bmi_{cat_val}'] = val
        
    # Age (Dekaden für Übersicht)
    df_risk_check['age_grp'] = pd.cut(df_risk_check['anchor_age'], bins=[0, 30, 50, 70, 90, 120], labels=['u30', '30_50', '50_70', '70_90', '90plus'])
    for cat_val, val in df_risk_check.groupby('age_grp', observed=True)['risk_score'].mean().items():
        risk_stats[f'risk_age_{cat_val}'] = val
    
    # Speichern
    result_dict = {
        'run_id': i, 'seed': current_seed, 'accuracy': acc, 'f1_macro': f1_macro,
        'precision_macro': prec_macro, 'recall_macro': rec_macro,
        'f1_class_0_early': f1_per_class[0], 'f1_class_1_late': f1_per_class[1], 'f1_class_2_healthy': f1_per_class[2],
        'duration_sec': duration
    }
    # Feature Importance hinzufügen
    for feat_name, importance_val in zip(feature_cols, perm_result.importances_mean):
        result_dict[f'imp_{feat_name}'] = importance_val
        
    # Risk-Stats hinzufügen
    result_dict.update(risk_stats)
    results_list.append(result_dict)

    # --- E. PLOTTING PRO RUN ---
    
    # 1. Confusion Matrix
    cm = confusion_matrix(y_val_robust, y_pred)
    cm_perc = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    labels = ['Früh', 'Spät', 'Gesund']
    annot_text = [[f"{val}<br>({perc:.1%})" for val, perc in zip(r, rp)] for r, rp in zip(cm, cm_perc)]
    fig_cm = ff.create_annotated_heatmap(cm_perc, x=labels, y=labels, annotation_text=annot_text, colorscale='Reds')
    fig_cm.update_layout(title=f'Confusion Matrix (Run {i})', height=400, width=500)
    fig_cm.write_image(os.path.join(run_dir, "confusion_matrix.png"))

    # 1b. Sankey Diagram
    label_list = ["Tatsächlich: Früh", "Tatsächlich: Spät", "Tatsächlich: Gesund", 
                  "Vorhergesagt: Früh", "Vorhergesagt: Spät", "Vorhergesagt: Gesund"]
    source = [0, 0, 0, 1, 1, 1, 2, 2, 2] 
    target = [3, 4, 5, 3, 4, 5, 3, 4, 5] 
    value = cm.flatten()
    color_link = [
        'rgba(31, 119, 180, 0.4)', 'rgba(31, 119, 180, 0.2)', 'rgba(31, 119, 180, 0.1)', # Von Früh
        'rgba(255, 127, 14, 0.2)', 'rgba(255, 127, 14, 0.4)', 'rgba(255, 127, 14, 0.1)', # Von Spät
        'rgba(44, 160, 44, 0.1)', 'rgba(44, 160, 44, 0.1)', 'rgba(44, 160, 44, 0.4)'    # Von Gesund
    ]
    fig_sankey = go.Figure(data=[go.Sankey(
        node = dict(pad = 15, thickness = 20, line = dict(color = "black", width = 0.5), label = label_list, color = "blue"),
        link = dict(source = source, target = target, value = value, color = color_link))])
    fig_sankey.update_layout(title_text=f"Patienten-Fluss (Run {i})", font_size=12, height=500)
    fig_sankey.write_image(os.path.join(run_dir, "sankey_flow.png"))
    
    # 2. Risk Plots Data Prep
    df_plot_run = X_val_robust.copy()
    df_plot_run['risk_score'] = y_proba[:, 0]
    
    # a) BMI
    bins_bmi = [0, 18.5, 25, 30, 100]
    labels_bmi = ['Untergewicht (<18.5)', 'Normal (18.5-25)', 'Übergewicht (25-30)', 'Adipositas (>30)']
    df_plot_run['bmi_group'] = pd.cut(df_plot_run['bmi'], bins=bins_bmi, labels=labels_bmi)
    df_bmi_agg = df_plot_run.groupby('bmi_group', observed=True)['risk_score'].mean().reset_index()
    
    fig_bmi = px.bar(df_bmi_agg, x='bmi_group', y='risk_score', text_auto='.1%', 
                     title=f'Risiko nach BMI-Gruppe (Run {i})', 
                     labels={'risk_score': 'Wahrscheinlichkeit', 'bmi_group': 'BMI Gruppe'},
                     template="plotly_white", height=400)
    fig_bmi.update_layout(yaxis_tickformat='.0%')
    fig_bmi.write_image(os.path.join(run_dir, "risk_bmi.png"))
    
    # b) Age
    bins_age = [0, 20, 30, 40, 50, 60, 70, 80, 90, 120]
    labels_age = ['<20', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80-89', '90+']
    df_plot_run['age_group'] = pd.cut(df_plot_run['anchor_age'], bins=bins_age, labels=labels_age)
    df_plot_run['age_group'] = df_plot_run['age_group'].astype(str)
    
    df_age_agg = df_plot_run.groupby('age_group')['risk_score'].mean().reset_index()
    
    fig_age = px.bar(df_age_agg, x='age_group', y='risk_score', text_auto='.1%', 
                     title=f'Risiko-Entwicklung über das Alter (Run {i})', 
                     labels={'risk_score': 'Wahrscheinlichkeit', 'age_group': 'Altersgruppe'},
                     template="plotly_white", height=400)
    fig_age.update_layout(yaxis_tickformat='.0%')
    fig_age.write_image(os.path.join(run_dir, "risk_age.png"))

    # c) Insurance & Gender
    df_ins_agg = df_plot_run.groupby(['insurance', 'gender'])['risk_score'].mean().reset_index()
    fig_ins = px.bar(df_ins_agg, x='insurance', y='risk_score', color='gender', barmode='group',
                     text_auto='.1%', 
                     title=f'Risiko: Versicherung & Geschlecht (Run {i})', 
                     labels={'risk_score': 'Wahrscheinlichkeit', 'insurance': 'Versicherung'},
                     template="plotly_white", height=400)
    fig_ins.update_layout(yaxis_tickformat='.0%')
    fig_ins.write_image(os.path.join(run_dir, "risk_insurance_gender.png"))

    # 3. Feature Importance 
    imp_df_run = pd.DataFrame({
        'Feature': feature_cols,
        'Importance': perm_result.importances_mean,
        'Std_Dev': perm_result.importances_std
    }).sort_values(by='Importance', ascending=True)

    fig_imp = px.bar(imp_df_run, x='Importance', y='Feature', orientation='h',
                     title=f'Feature Importance (Run {i})',
                     labels={'Importance': 'Wichtigkeit (Mean Decrease Accuracy)'}, error_x='Std_Dev',
                     template="plotly_white", color='Importance', color_continuous_scale='Reds')
    # NEU: Auch innerhalb der Runs Prozent-Formatierung für die Bilder
    fig_imp.update_layout(height=400, showlegend=False, xaxis_tickformat='.1%')
    fig_imp.write_image(os.path.join(run_dir, "feature_importance.png"))

    if i == 0 or (i+1) % 5 == 0:
        print(f"   Run {i+1}/{N_LOOPS}: F1={f1_macro:.3f} | Bilder in {run_dir}/")

# 3. Speichern
df_results = pd.DataFrame(results_list)
csv_path = os.path.join(RESULTS_DIR, "robustness_metrics_with_importance.csv")
df_results.to_csv(csv_path, index=False)
print(f"✅ Experiment abgeschlossen. Daten gespeichert in: {csv_path}")

⚠️ TEST-LAUF: Starte nur 1 Runde.
🚀 Starte Experiment. Ergebnisse landen in: ../../reports/robustness_experiment_test/2026-02-14_15-35-56


/Users/andrey/Repositories/fs-thesis/.venv/lib/python3.13/site-packages/tabpfn/validation.py:56: UserWarning:

Running on CPU with more than 200 samples may be slow.
Consider using a GPU or the tabpfn-client API: https://github.com/PriorLabs/tabpfn-client



   Run 1/1: F1=0.318 | Bilder in ../../reports/robustness_experiment_test/2026-02-14_15-35-56/run_0/
✅ Experiment abgeschlossen. Daten gespeichert in: ../../reports/robustness_experiment_test/2026-02-14_15-35-56/robustness_metrics_with_importance.csv


In [ ]:
# 4. Visualisierung der Robustness (Master-Thesis Style)
# Ziel: Durchschnittliche Performance zeigen + Stabilität (Fehlerbalken) beweisen

# Melt transformiert die Daten für Plotly (Wide -> Long Format)
df_melt = df_results.melt(
    id_vars=['run_id'], 
    value_vars=['f1_class_0_early', 'f1_class_1_late', 'f1_class_2_healthy'],
    var_name='Target Class', 
    value_name='F1 Score'
)

# Namen verschönern
df_melt['Target Class'] = df_melt['Target Class'].replace({
    'f1_class_0_early': 'Früher Ausbruch (<1J)',
    'f1_class_1_late': 'Später Ausbruch (1-3J)', 
    'f1_class_2_healthy': 'Kein Event / Gesund'
})

# Aggegierte Statistiken für Bar-Chart
df_stats = df_melt.groupby('Target Class')['F1 Score'].agg(['mean', 'std']).reset_index()

# Sortierung manuell festlegen (Logische Zeitreihe)
category_order = ['Früher Ausbruch (<1J)', 'Später Ausbruch (1-3J)', 'Kein Event / Gesund']

# 1. Bar Chart mit Error Bars (Klassisch wissenschaftlich)
fig = px.bar(
    df_stats, 
    x="Target Class", 
    y="mean", 
    error_y="std", # Zeigt die Standardabweichung als Antenne (Robustheit)
    title=f"Modell-Performance: Durchschnitt & Stabilität ({N_LOOPS} Runs)",
    text_auto='.1%', # Beschriftung direkt am Balken
    labels={'mean': 'Durchschnittlicher F1-Score'},
    color="Target Class", 
    # Pastell Farben wirken professioneller und weniger überladen
    color_discrete_sequence=px.colors.qualitative.Pastel,
    template="plotly_white",
    category_orders={"Target Class": category_order} # Erzwingt die logische zeitliche Reihenfolge
)

# Balken leicht transparent machen (0.8), damit sie nicht zu massiv wirken
fig.update_traces(marker_opacity=0.8, showlegend=False)

# 2. Scatter-Punkte darüber legen (Kontrastfarbe für bessere Sichtbarkeit)
# Zeigt jeden einzelnen Run als Punkt -> Maximale Transparenz der Ergebnisse
scatter_trace = px.strip(
    df_melt, 
    x="Target Class", 
    y="F1 Score", 
    category_orders={"Target Class": category_order}
).data[0]

# Styling der Punkte: Dunkles Kontrast-Blau mit weißem Rand
# Das sorgt für Lesbarkeit sowohl auf hellen als auch dunklen Hintergründen
scatter_trace.marker.color = '#34495e' # "Wet Asphalt" (Dunkles Grau-Blau)
scatter_trace.marker.size = 6
scatter_trace.marker.opacity = 0.8
scatter_trace.marker.line = dict(width=1, color='white') # Weißer Rand lässt Punkte "poppen"
scatter_trace.showlegend = False

fig.add_trace(scatter_trace)

# Achsen formatieren (Prozent statt 0.x)
fig.update_layout(
    yaxis_tickformat='.0%', 
    yaxis_title="F1 Score (Macro)", 
    xaxis_title=None, # X-Achsen Titel ist redundant wegen den Labels
    font=dict(size=14) # Schrift etwas größer für Thesis
)
fig.update_yaxes(range=[0, 1.1]) # Platz für Error Bars lassen

fig.write_image(os.path.join(RESULTS_DIR, "robustness_scientific_bar.png"))
fig.show()

In [ ]:
# 4b. Visualisierung der Feature Importance Stabilität
import pandas as pd
import plotly.express as px
import os

# Melt für Feature Wichtigkeit
feature_imp_cols = [c for c in df_results.columns if c.startswith("imp_")]
df_melt_imp = df_results.melt(
    id_vars=['run_id'], 
    value_vars=feature_imp_cols,
    var_name='Feature', 
    value_name='Importance Score'
)

# Prefix "imp_" für schönere Labels entfernen
df_melt_imp['Feature'] = df_melt_imp['Feature'].str.replace('imp_', '')

# Sortierung berechnen: Wir wollen das wichtigste Feature OBEN haben.
# Plotly Boxplots ordern die Y-Achse standardmäßig von unten nach oben.
# Wir sortieren hier nach Median aufsteigend (ascending=True).
# -> Das Feature mit dem kleinsten Median steht am Anfang der Liste (Index 0).
# -> Plotly zeichnet Index 0 unten.
# -> Das Feature mit dem größten Median steht am Ende der Liste.
# -> Plotly zeichnet das Ende oben.
# Falls es bei dir "falsch herum" ist, liegt es oft daran, dass Plotly 'Feature' noch als Faktor hat.
sorted_features = df_melt_imp.groupby('Feature')['Importance Score'].median().sort_values(ascending=False).index.tolist()

fig = px.box(
    df_melt_imp, 
    x="Importance Score", 
    y="Feature", 
    color="Feature", 
    # Wir erzwingen die Sortierung explizit über category_orders
    category_orders={"Feature": sorted_features}, 
    title=f"Feature Wichtigkeit über {N_LOOPS} Runs (Stabilitätstest)",
    points="all", 
    template="plotly_white",
    height=600,
    color_discrete_sequence=px.colors.qualitative.Pastel
)

fig.update_layout(showlegend=True)

# Punkte styling 
fig.update_traces(marker=dict(opacity=0.6, size=3)) # color='#34495e', 

# Achsen formatieren
fig.update_layout(
    xaxis_tickformat='.1%', 
    xaxis_title="Wichtigkeit (Mean Decrease Accuracy)",
    yaxis_title=None,
    font=dict(size=12)
)

# Speichern
fig.write_image(os.path.join(RESULTS_DIR, "robustness_feature_importance.png"))
fig.show()

In [ ]:
# 5. Sanity Check / Plausibilitäts-Prüfung
# Hinweis: RESULTS_DIR kommt jetzt aus dem Block oben mit Zeitstempel!

import pandas as pd
import os

print(f"Lade Prüfung von: {RESULTS_DIR}") # Nimmt den aktuellsten Ordner
try:
    csv_load_path = os.path.join(RESULTS_DIR, "robustness_metrics_with_importance.csv")
    check_df = pd.read_csv(csv_load_path)

    print(f"Anzahl durchgeführter Runs: {len(check_df)}")
    print(f"Genutzte Seeds: {check_df['seed'].unique()}")

    # Prüfung 1: Haben wir Variation?
    if len(check_df) > 1:
        std_dev = check_df['f1_macro'].std()
        if std_dev > 0:
            print(f"✅ Test bestanden: Ergebnisse variieren (Std-Dev: {std_dev:.4f}).")
        else:
            print("⚠️ Warnung: Keine Variation! Prüfe den Random Seed!")
    else:
        print("ℹ️ Hinweis: Nur 1 Run durchgeführt (Test-Modus).")

    # Prüfung 2: Sind die Ergebnisse 'zu perfekt'?
    if check_df['f1_macro'].max() >= 0.99:
        print("⚠️ Warnung: F1-Score von fast 1.0 (>=0.99) gefunden. Das ist verdächtig.")
    else:
        print(f"✅ Test bestanden: Realistische Scores (Max: {check_df['f1_macro'].max():.3f})")

    check_df.head()
    
except FileNotFoundError:
    print(f"❌ Fehler: Keine CSV gefunden in {RESULTS_DIR}. Hast du den Loop oben laufen lassen?")

Lade Prüfung von: ../../reports/robustness_experiment_test/2026-02-14_15-35-56
Anzahl durchgeführter Runs: 1
Genutzte Seeds: [42]
ℹ️ Hinweis: Nur 1 Run durchgeführt (Test-Modus).
✅ Test bestanden: Realistische Scores (Max: 0.318)
